In [ ]:
## Setup:
import zipfile
import requests
import pandas as pd
import os
date=[];    data=[];    start=1993;     end=2019
numy=end-start+1;       numq=numy*4
quart=["0331","0630","0930","1231"]
path="https://www7.fdic.gov/sdi/Resource/AllReps/All_Reports_"
direct=os.getcwd()+"\\"
direct = direct.replace('\\','/')
name="All_Reports_"

## Pulling data:
Balance=[];    balfile="_Assets and Liabilities.csv"
Income=[];     incfile="_Income and Expense.csv"
for i in range(numy):
    for j in range(len(quart)):
        date=str(i+start)+quart[j]
        url=path+date+".zip"
        resp=requests.get(url)
        output=open('data.zip','wb')
        output.write(resp.content)
        output.close()
        zip_ref=zipfile.ZipFile(direct+"data.zip")
        zip_ref.extract(name+date+balfile,direct)
        Balance.append(pd.read_csv(direct+name+date+balfile,usecols=['cert',
                    'repdte','rssdhcr','offdom','numemp','asset',
                    'chbal','sc','lnlsnet','lnatres','bkprem','ore',
                    'intan','idoa','liab','dep','eqtot', 'eq','RWAJT',
                    'RBCT1J','uc','obsdir'],error_bad_lines=False,low_memory=False))
        zip_ref.extract(name+date+incfile,direct)
        Income.append(pd.read_csv(direct+name+date+incfile,usecols=['cert',
                    'repdte','intinc','eintexp','elnatr','nonii','ifiduc',
                    'iserchg','igltrad','nonix','esal','epremagg','iglsec',
                    'itax','extra','netinc','eqcdiv',
                    'eqcstkrx'],error_bad_lines=False,low_memory=False))
        os.remove(direct+name+date+balfile)
        os.remove(direct+name+date+incfile)
## Creating files:
zip_ref.close()
os.remove(direct+"data.zip")
balance=Balance[0]; income=Income[0]
for i in range(1,numq):
    balance=pd.concat([balance,Balance[i]])
    income=pd.concat([income,Income[i]])
balance.to_csv("../../data/2020_12/balance.csv")
income.to_csv("../../data/2020_12/income.csv")

type(balance)
pd.unique(balance['cert']).size

###############################################################################
## Data Cleaning:
###############################################################################

## Pulling in the data previously generated:
import pandas as pd
from numpy import maximum
import requests
balance=pd.read_csv("balance.csv")
income=pd.read_csv("income.csv")

## Creating proper date variable:
balance["date"]=pd.to_datetime(balance["repdte"], infer_datetime_format=True)
income["date"]=pd.to_datetime(income["repdte"], infer_datetime_format=True)
data=pd.merge(balance,income,how="inner",on=["cert","date"])

## Merging in bank failures data:
urlfail="https://pfabankapi.app.cloud.gov/api/failures?fields=NAME%2CCERT%2CFIN%2CCITYST%2CFAILDATE%2CSAVR%2CRESTYPE%2CCOST%2CRESTYPE1%2CCHCLASS1%2CQBFDEP%2CQBFASSET&filters=FAILYR%3A%5B1934%20TO%202017%5D&limit=10000&react=true&sort_by=FAILDATE&sort_order=desc&subtotal_by=RESTYPE&total_fields=QBFDEP%2CQBFASSET%2CCOST&format=csv&download=true&filename=bank-data"
resp=requests.get(urlfail)
output=open('fails.csv','wb')
output.write(resp.content)
output.close()
fails=pd.read_csv("fails.csv")
fails=fails[["CERT","FAILDATE","RESTYPE"]]
fails["faildate"]=pd.to_datetime(fails["FAILDATE"], infer_datetime_format=True)
fails["cert"]=fails["CERT"]
fails['date']=[date - pd.tseries.offsets.DateOffset(days=1) - pd.tseries.offsets.QuarterEnd() for date in  fails["faildate"]]
fails["fail"]=1
fails["year"]=pd.DatetimeIndex(fails["date"]).year
fails=fails[fails["year"]>1992]
fails=fails[["cert","date","fail"]]
data=pd.merge(data,fails,how='left',on=["cert","date"])
data["fail"] = data["fail"].fillna(0)
os.remove(direct+"fails.csv")

## Combining holding companies:
data["ID"]=balance[["cert", "rssdhcr"]].max(axis=1)
data["ID"]=data["ID"].astype('int64')
data=data.drop(["cert","rssdhcr"],axis=1)
data=data.groupby([data["ID"],data["date"]]).sum()
data=data.reset_index()

# Converting income statement variabes into changes:
data=data.sort_values(by=["ID","date"])
data["first"]=data["ID"]!=data["ID"].shift(1)
data["year"]=pd.DatetimeIndex(data["date"]).year
data["month"]=pd.DatetimeIndex(data["date"]).month
data["march"]=data["month"]==3
data["flow"]=maximum(data["first"],data["march"])
inc_cols=income.columns.drop(["cert","date"])
for i in inc_cols:
    name=i+"_diff"
    data[name]=data[i]-data[i].shift(1)
    data[name] = data[name].fillna(0)
    data[i]=data[i]*data["flow"]+data[name]*(1-data["flow"])
    data=data.drop([name],axis=1)

## Saving file:
# data=data.drop(["first","year","month","march","flow"],axis=1)
data.to_csv("../../data/2020_12/data_use.csv")